In [ ]:
# import joblib
import pandas as pd
import tensorflow as tf


# model1 = tf.keras.models.load_model('./CNN_second_model.h5')
model2 = tf.keras.models.load_model('/content/drive/MyDrive/CNN_second_model2.h5')
# transformer = joblib.load('corpus_transformer.pkl') 

# model = joblib.load('CNN_best_model3.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import itertools
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
from glob import glob
plt.style.use('seaborn-white')

In [ ]:
# path = glob(os.path.join('녹음.wav'))

In [ ]:
# path

In [ ]:
def get_target_data(data_dir):
    audio_path_list = []
    audio_path_list.extend(glob(os.path.join(data_dir,'*.wav')))
    label_list = [1] * len(audio_path_list)
           
    return audio_path_list, label_list

In [ ]:
import os
from glob import glob
all_audio_path, all_label = get_target_data('C:/Users/user/Desktop/KB국민 아카데미/머신러닝/원천데이터/sample')

In [ ]:
len(all_audio_path)

1073

In [ ]:
def get_normal_data(data_dir):
    normal_path_list = []
    
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        for case_name_2 in os.listdir(current_path):
            current_path_2 = os.path.join(current_path, case_name_2)
            for case_name_3 in os.listdir(current_path_2):
                current_path_3 = os.path.join(current_path_2, case_name_3)
                normal_path_list.extend(glob(os.path.join(current_path_3,'*.wav')))
    
    label_list = [0] * len(normal_path_list)
           
    return normal_path_list, label_list

In [ ]:
all_normal_path, all_normal_label = get_normal_data('C:/Users/user/Desktop/KB국민 아카데미/머신러닝/원천데이터/KrespSpeech')

In [ ]:
len(all_normal_path)

5463

In [ ]:
# path = '/content/drive/MyDrive/실제음성_데이터/0001.wav'
# path = 'C:\\Users\\user\\Desktop\\KB\\data\\정상.wav'
path = "blob:http://127.0.0.1:8000/39067dcc-d1a3-456f-a1c5-8c5383c8e413.wav"
y, sr = librosa.load(path)
n_fft = int(np.ceil(0.025 * sr))
win_length = int(np.ceil(0.025 * sr))
hop_length = int(np.ceil(0.01 * sr))

S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40, win_length=win_length, hop_length=hop_length, n_fft=n_fft)


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


FileNotFoundError: ignored

In [ ]:
S.shape

(40, 215)

In [ ]:
arr_df = []

In [ ]:
arr_df.append( list(itertools.chain(*S)))

In [ ]:
avg = sum(arr_df[0])/len(arr_df[0])
avg

0.009540002857029717

In [ ]:
l = len(arr_df[0])
l

8600

In [ ]:
# 파일 길이가 max_num 길이보다 긴 경우 max_num으로 나눠서 모든 섹션을 확인하여 더해준다.

max_num = 14960
arr_re = []

pred_now = 0
idx = l//max_num
for i in range(idx):
  arr_re = []
  for j in range(i*max_num,(i+1)*max_num):
    arr_re.append(arr_df[0][j])

  padded_np = np.array(arr_re)

  b = padded_np.reshape(1, 40, 374)

  re_np = np.expand_dims(b, -1)

  pred = model2.predict(re_np)
  print(pred)
  pred_now += pred

if l%max_num != 0:
  arr_re = []
  for i in range(idx*max_num,l):
    arr_re.append(arr_df[0][i])
  for i in range(max_num-l%max_num):
        arr_re.append(avg)
  padded_np = np.array(arr_re)

  b = padded_np.reshape(1, 40, 374)

  re_np = np.expand_dims(b, -1)

  pred = model2.predict(re_np)
  print(pred)
  pred_now += pred

print(pred_now)
if pred_now > 0.1:
  print(1)
else:
  print(0)

[[0.00028694]]
[[0.51045406]]
[[0.510741]]
1


In [ ]:
# 파일 길이가 max_num보다 짧은 경우 avg 값을 뒤에 추가하여 길이를 맞춰준다.

max_num = 14960
arr_re = []

for i in range(l):
  arr_re.append(arr_df[0][i])
for i in range(max_num-l):
      arr_re.append(avg)

padded_np = np.array(arr_re)

b = padded_np.reshape(1, 40, 374)

re_np = np.expand_dims(b, -1)

pred = model2.predict(re_np)
print(pred)
if np.round(pred)[0][0] > 0.1:
  print(1)
else:
  print(0)

[[0.00600991]]
0


In [ ]:
print(len(arr_re))

14960


In [ ]:
# for sentence in arr_re:
#     while len(sentence) < 19920:
#         sentence.append(0)

# padded_np = np.array(arr_df)
# padded_np

In [ ]:
padded_np = np.array(arr_re)
padded_np

array([1.10173153e+00, 1.10173153e+00, 1.10173153e+00, ...,
       2.29279703e-06, 4.28097792e-06, 2.11804709e-05])

In [ ]:
b = padded_np.reshape(1, 80, 520)

In [ ]:
re_np = np.expand_dims(b, -1)
re_np.shape

(1, 80, 520, 1)

In [ ]:
pred = model2.predict(re_np)
pred

array([[1.]], dtype=float32)

In [ ]:
np.round(pred)[0][0]

1.0

In [ ]:
count = 0
count_less = 0
count_more = 0
count_less2 = 0
count_more2 = 0
for path in all_audio_path:
  y, sr = librosa.load(path)
  n_fft = int(np.ceil(0.025 * sr))
  win_length = int(np.ceil(0.025 * sr))
  hop_length = int(np.ceil(0.01 * sr))

  S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40, win_length=win_length, hop_length=hop_length, n_fft=n_fft)

  arr_df = []
  arr_df.append(list(itertools.chain(*S)))

  l = len(arr_df[0])
  
  arr_re = []

  avg = sum(arr_df[0])/l
  
  if l < 14960:
    count_less += 1
    for i in range(l):
          arr_re.append(arr_df[0][i])
    for i in range(14960-l):
          arr_re.append(avg)
    padded_np = np.array(arr_re)

    b = padded_np.reshape(1, 40, 374)

    re_np = np.expand_dims(b, -1)

    pred = model2.predict(re_np)

    if np.round(pred)[0][0] > 0.1:
      count_less2 += 1
  else:
    count_more += 1
    pred_now = 0
    idx = l//14960
    for i in range(idx):
      arr_re = []
      for j in range(i*14960,(i+1)*14960):
        arr_re.append(arr_df[0][j])
      padded_np = np.array(arr_re)

      b = padded_np.reshape(1, 40, 374)

      re_np = np.expand_dims(b, -1)

      pred = model2.predict(re_np)
      pred_now += np.round(pred)[0][0]
    
    if l%14960 !=0:
      arr_re = []
      for i in range(idx*14960,l):
          arr_re.append(arr_df[0][i])
      for i in range(14960-l%14960):
          arr_re.append(avg)
      padded_np = np.array(arr_re)

      b = padded_np.reshape(1, 40, 374)

      re_np = np.expand_dims(b, -1)

      pred = model2.predict(re_np)
      pred_now += np.round(pred)[0][0]

    if pred_now > 0.1:
      count_more2 += 1

  # print(np.round(pred)[0][0])

In [ ]:
print(count_less2 / count_less, count_more2 / count_more, (count_more2 + count_less2) / ( count_less + count_more))

0.8564954682779456 0.5158150851581509 0.7260018639328985
